# Build and test the otsu container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/otsu/

## Goal

Create a container image and run the otsu step in a container.


## Setup the environment

In [2]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [3]:
cat ${WORKSPACE}/water-bodies/command-line-tools/otsu/Dockerfile

FROM docker.io/python:3.10-slim

RUN pip install --no-cache-dir rasterio scikit-image click loguru && \
    python -c "import rasterio"

ADD app.py /app/app.py

ENTRYPOINT []


Build the container using `podman`:

In [6]:
podman build --format docker -t localhost/otsu:latest ${WORKSPACE}/water-bodies/command-line-tools/otsu


STEP 1/4: FROM docker.io/python:3.10-slim
STEP 2/4: RUN pip install --no-cache-dir rasterio scikit-image click loguru &&     python -c "import rasterio"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53

Show the `otsu` help:

In [7]:
podman run --rm -it --env=PYTHONPATH=/app localhost/otsu:latest python -m app --help

Usage: app.py [OPTIONS] RASTER

  Applies the Otsu threshold

Options:
  --help  Show this message and exit.


## Test the otsu step in the container

Generate the otsu automatic threshold on the normalized difference:

In [8]:
podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/norm_diff.tif,target=/inputs/norm_diff.tif,readonly \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/otsu:latest \
    python \
    -m \
    app \
    /inputs/norm_diff.tif

2024-04-11 13:46:18.698 | INFO     | __main__:otsu:37 - Applying the Otsu threshold to /inputs/norm_diff.tif
2024-04-11 13:46:18.702 | INFO     | __main__:otsu:40 - Write otsu.tif
2024-04-11 13:46:20.426 | INFO     | __main__:otsu:43 - Done!


List the outputs:

In [9]:
tree ${RUNTIME}

/workspace/mastering-app-package/runs
├── crop_green.tif
├── crop_nir.tif
├── norm_diff.tif
└── otsu.tif

0 directories, 4 files
